# Desafio de estágio em IA

## GAN

Nesta seção do código, está sendo utilizado um tipo especial de modelo de aprendizado de máquina chamado Rede Generativa Adversarial (GAN) para gerar previsões futuras. GANs consistem em um gerador e um discriminador que trabalham juntos para criar dados sintéticos realistas. Nesse contexto, o gerador gera amostras de dados futuros com base em um espaço de latente aleatório, enquanto o discriminador avalia a autenticidade dessas amostras.

No início do código, a arquitetura do gerador e do discriminador é definida usando camadas densas da biblioteca Keras. O gerador recebe um vetor de latente como entrada e gera previsões de valores futuros. O discriminador, por sua vez, recebe amostras reais (dados históricos) e amostras geradas pelo gerador, atribuindo uma probabilidade de autenticidade a cada uma delas.

Durante o treinamento da GAN, várias épocas são percorridas e os pesos do gerador e do discriminador são atualizados com base nas perdas calculadas. Após o treinamento, previsões futuras são geradas usando o gerador. Essas previsões são normalizadas e desnormalizadas para obter os valores reais em termos de preços de fechamento do mercado de ações.

### Importações das bibliotecas usadas

Nesse trecho de código, são importadas algumas bibliotecas e módulos necessários para o projeto.

- `numpy` (importado como `np`) é uma biblioteca popular para computação numérica em Python, oferecendo suporte a arrays multidimensionais e uma ampla variedade de funções matemáticas.
- `pandas` (importado como `pd`) é uma biblioteca usada para manipulação e análise de dados. Ela fornece estruturas de dados poderosas, como o DataFrame, que facilitam o trabalho com conjuntos de dados tabulares.
- `tensorflow` (importado como `tf`) é uma biblioteca de código aberto amplamente utilizada para aprendizado de máquina e inteligência artificial. Ela oferece uma variedade de ferramentas e funcionalidades para construir, treinar e implantar modelos de aprendizado de máquina.
- `tensorflow.keras.layers` é um módulo do TensorFlow que fornece uma API para criar camadas de redes neurais em modelos de aprendizado profundo.
- `backtesting` é uma biblioteca que facilita o teste e a avaliação de estratégias de negociação em dados históricos de mercado. Ela fornece classes e métodos para realizar backtesting de estratégias, avaliar métricas de desempenho e visualizar os resultados.

Essas importações são feitas para utilizar as funcionalidades dessas bibliotecas e módulos ao longo do código.

In [47]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import layers, regularizers
from backtesting import Backtest, Strategy


### Leitura do CSV

Nesse trecho de código, é utilizada a biblioteca pandas para ler um arquivo CSV contendo dados relacionados ao preço do petróleo em 2021. A função `pd.read_csv()` é usada para carregar os dados do arquivo CSV e armazená-los em um objeto do tipo DataFrame. Em seguida, é selecionada apenas a coluna "Close" do DataFrame original e atribuída novamente à variável `dados`. Essa coluna representa os preços de fechamento do petróleo. Ao fazer isso, os dados são filtrados para utilizar apenas essa informação específica em análises ou processamentos posteriores. Essa operação é comum quando se deseja trabalhar com uma variável específica de um conjunto de dados mais amplo.

In [48]:
dados = pd.read_csv('dados_petroleo_2021.csv')
dados = dados['Close']  

### Normalização dos dados

Nesse trecho de código, é realizada a normalização dos dados. Primeiro, é subtraída a média dos dados (`dados.mean()`) de cada valor do conjunto de dados `dados`. Em seguida, o resultado é dividido pelo desvio padrão dos dados (`dados.std()`). Esse processo de normalização é comumente usado para colocar os dados em uma escala com média zero e desvio padrão igual a um. A normalização dos dados é útil em várias técnicas de análise e modelagem, pois ajuda a lidar com diferenças de escala entre as variáveis e permite compará-las de maneira mais adequada.

In [49]:
dados_normalizados = (dados - dados.mean()) / dados.std()

### Hiperparâmetros da GAN

Comentário sobre os Hiperparâmetros da GAN:

Neste trecho de código, são definidos os hiperparâmetros da Rede Generativa Adversarial (GAN). Os hiperparâmetros são valores configuráveis que afetam o comportamento e o desempenho do modelo.

- A variável `dimensao_latente` representa a dimensão do vetor latente utilizado como entrada para o gerador da GAN. Essa dimensão determina a complexidade e a variabilidade das amostras sintéticas geradas pelo modelo. No caso desse código, o vetor latente tem dimensão 10.

- A variável `epocas` indica o número de épocas de treinamento da GAN. Uma época corresponde a uma iteração completa sobre todo o conjunto de dados. Quanto maior o número de épocas, mais oportunidades o modelo tem para aprender e melhorar suas previsões. Neste caso, são definidas 150 épocas de treinamento.

- A variável `tamanho_lote` especifica o tamanho do lote de dados utilizado em cada iteração durante o treinamento da GAN. Um lote é um conjunto de amostras processadas simultaneamente pelo modelo antes de atualizar os pesos. O tamanho do lote pode afetar a estabilidade do treinamento e o uso eficiente dos recursos computacionais. Neste código, o tamanho do lote é definido como 128.

Esses hiperparâmetros podem ser ajustados de acordo com as necessidades específicas do problema e podem impactar o desempenho e a qualidade das previsões geradas pela GAN.

In [50]:
dimensao_latente = 10
epocas = 150
tamanho_lote = 128

### Gerador

Nesse trecho de código, é definido um gerador da GAN, que é uma rede neural artificial sequencial composta por camadas densas (fully connected layers). A primeira camada possui 256 neurônios com função de ativação ReLU e recebe como entrada um vetor de dimensão_latente. A segunda camada possui 512 neurônios com função de ativação ReLU. A última camada possui um único neurônio com função de ativação linear, que produzirá a saída do gerador. Essa arquitetura define como um gerador indeterminado irá transformar o vetor latente em previsões futuras.

In [51]:
gerador = tf.keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(dimensao_latente,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='linear')
])

### Discriminador

Nesse trecho de código, é definido um discriminador da GAN, que também é uma rede neural artificial sequencial composta por camadas densas. A primeira camada possui 512 neurônios com função de ativação ReLU e recebe como entrada um único valor. A segunda camada possui 256 neurônios com função de ativação ReLU. A última camada possui um único neurônio com função de ativação sigmoid, que produzirá a saída do discriminador. Essa arquitetura define como um discriminador indeterminado irá classificar se uma entrada é real ou gerada por um gerador indeterminado.

In [52]:
discriminador = tf.keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(1,)),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

### Função de perda e otimizadores

Neste trecho de código, são definidas três variáveis:

1. `funcao_perda` é uma instância da classe `BinaryCrossentropy` do módulo `tf.keras.losses`. Essa função de perda é comumente utilizada em problemas de classificação binária para calcular a diferença entre as previsões e os rótulos verdadeiros.
2. `otimizador_gerador` é uma instância da classe `Adam` do módulo `tf.keras.optimizers`. Esse otimizador é responsável por atualizar os pesos do gerador durante o treinamento da GAN. A taxa de aprendizado (learning rate) definida é de 0.0001.
3. `otimizador_discriminador` é outra instância da classe `Adam` do módulo `tf.keras.optimizers`. Esse otimizador é responsável por atualizar os pesos do discriminador durante o treinamento da GAN. Assim como o `otimizador_gerador`, a taxa de aprendizado definida é de 0.0001.

Essas variáveis são essenciais para configurar a função de perda e os otimizadores que serão utilizados na GAN, contribuindo para a aprendizagem dos modelos gerador e discriminador.

In [53]:
funcao_perda = tf.keras.losses.BinaryCrossentropy()
otimizador_gerador = tf.keras.optimizers.Adam(learning_rate=0.0001)
otimizador_discriminador = tf.keras.optimizers.Adam(learning_rate=0.0001)

### Função de treinamento da GAN

Neste trecho de código, é definida a função `treinar_gan`, decorada com `@tf.function`, indicando que ela será compilada em um grafo TensorFlow para otimização de desempenho. A função recebe `dados_reais` como entrada.

A função realiza as seguintes etapas:

1. Obtém o tamanho do lote a partir das dimensões dos `dados_reais`.
2. Gera um vetor latente aleatório usando `tf.random.normal`, com o tamanho do lote e a dimensão latente desejada.
3. Cria dois blocos de contexto com `tf.GradientTape()`, um para o gerador (`gerador_tape`) e outro para o discriminador (`discriminador_tape`).
4. Dentro dos blocos de contexto, o gerador é utilizado para gerar dados sintéticos a partir do vetor latente.
5. O discriminador é aplicado tanto aos dados reais quanto aos dados gerados para obter os resultados (`resultado_dados_reais` e `resultado_dados_gerados`, respectivamente).
6. Calcula a perda do discriminador utilizando a função de perda (`funcao_perda`), somando a perda para os dados reais e a perda para os dados gerados.
7. Calcula a perda do gerador utilizando a função de perda, considerando apenas os resultados dos dados gerados.
8. Calcula os gradientes da perda em relação às variáveis treináveis do gerador e do discriminador, respectivamente.
9. Aplica os gradientes aos otimizadores do gerador e do discriminador utilizando `apply_gradients`, atualizando os pesos das redes neurais.

Em resumo, esse código implementa um passo de treinamento de uma rede GAN (Generative Adversarial Network), em que o gerador e o discriminador são treinados alternadamente para melhorar a qualidade dos dados gerados pelo gerador e a capacidade do discriminador em distinguir dados reais dos dados gerados.

In [54]:
@tf.function
def treinar_gan(dados_reais):
    tamanho_lote = dados_reais.shape[0]
    vetor_latente = tf.random.normal([tamanho_lote, dimensao_latente])

    with tf.GradientTape() as gerador_tape, tf.GradientTape() as discriminador_tape:
        dados_gerados = gerador(vetor_latente)

        resultado_dados_reais = discriminador(dados_reais)
        resultado_dados_gerados = discriminador(dados_gerados)

        perda_discriminador = funcao_perda(tf.ones_like(resultado_dados_reais), resultado_dados_reais) + \
            funcao_perda(tf.zeros_like(resultado_dados_gerados), resultado_dados_gerados)
        perda_gerador = funcao_perda(tf.ones_like(resultado_dados_gerados), resultado_dados_gerados)

    gradientes_gerador = gerador_tape.gradient(perda_gerador, gerador.trainable_variables)
    gradientes_discriminador = discriminador_tape.gradient(perda_discriminador, discriminador.trainable_variables)

    otimizador_gerador.apply_gradients(zip(gradientes_gerador, gerador.trainable_variables))
    otimizador_discriminador.apply_gradients(zip(gradientes_discriminador, discriminador.trainable_variables))

### Treinamento da GAN

Neste trecho de código, é realizado um loop de treinamento ao longo de um número especificado de épocas.

Dentro de cada iteração do loop, os seguintes passos são executados:

1. É gerado um índice aleatório usando `np.random.randint` para selecionar uma parte aleatória dos dados normalizados.
2. Os dados reais são extraídos do conjunto de dados normalizados com base no índice aleatório gerado. Em seguida, os dados são remodelados para terem uma dimensão de (-1, 1).
3. A função `treinar_gan` é chamada, passando os dados reais como entrada para treinar o modelo GAN.

Após o treinamento em cada iteração, é verificado se o número da época atual é um múltiplo de 100 usando o operador `%`. Se a condição for verdadeira, uma mensagem indicando a época atual e o número total de épocas é exibida.

Em resumo, este código implementa um loop de treinamento para uma GAN, onde a cada época um lote aleatório de dados é selecionado e o modelo é treinado utilizando esses dados. A cada 100 épocas, uma mensagem de progresso é exibida.

In [55]:
for epoca in range(epocas):
    indice_aleatorio = np.random.randint(0, len(dados_normalizados) - tamanho_lote)
    dados_reais = dados_normalizados[indice_aleatorio:indice_aleatorio + tamanho_lote].values.reshape(-1, 1)

    treinar_gan(dados_reais)

    if epoca % 10 == 0:
        print(f'Época {epoca}/{epocas}')


Época 0/150
Época 10/150
Época 20/150
Época 30/150
Época 40/150
Época 50/150
Época 60/150
Época 70/150
Época 80/150
Época 90/150
Época 100/150
Época 110/150
Época 120/150
Época 130/150
Época 140/150


### Gerar previsões futuras

Neste trecho de código, é gerado um vetor latente aleatório utilizando `tf.random.normal`. O vetor latente tem uma forma de `[tamanho_lote, dimensao_latente]`, em que `tamanho_lote` representa o número de amostras e `dimensao_latente` é a dimensão do vetor latente.

Em seguida, o gerador é utilizado para gerar previsões futuras com base nesse vetor latente. As previsões futuras são obtidas chamando a função `gerador` e passando o vetor latente como entrada.

Resumindo, esse código gera um vetor latente aleatório e utiliza o gerador para produzir previsões futuras com base nesse vetor latente.

In [56]:
vetor_latente = tf.random.normal([tamanho_lote, dimensao_latente])
previsoes_futuras = gerador(vetor_latente)

### Desnormalizar as previsões

Neste trecho de código, a variável `previsoes_futuras` é convertida em um array NumPy usando o método `numpy()`. Em seguida, uma sequência de operações é aplicada para reverter a normalização que foi aplicada aos dados.

Primeiro, é utilizado o método `reshape(-1)` para remodelar o array, transformando-o em um vetor unidimensional.

Em seguida, é realizada a reversão da normalização nos valores do vetor `previsoes_futuras`. Isso é feito multiplicando o vetor pelos desvios padrão dos dados originais (`dados.std()`) e adicionando a média dos dados originais (`dados.mean()`).

Por fim, o vetor `previsoes_futuras` é impresso utilizando a função `print()`.

Em resumo, esse código desfaz a normalização aplicada às previsões futuras, levando em consideração a média e o desvio padrão dos dados originais, e imprime o resultado obtido.

In [57]:
previsoes_futuras = previsoes_futuras.numpy().reshape(-1) * dados.std() + dados.mean()

print(previsoes_futuras)

[22.586512 22.460976 20.365696 20.980324 22.805073 22.310526 22.849524
 21.273417 23.626001 23.366646 24.057444 21.652637 20.913473 20.79905
 23.118414 22.285988 23.279182 20.630163 22.523676 21.388988 27.719238
 23.657763 23.214386 22.128666 21.117252 22.528849 22.236298 23.583553
 21.007446 21.263449 22.198153 22.700161 22.973318 22.625158 23.275957
 21.24421  21.980797 22.710535 23.123531 21.752947 22.381626 24.303997
 21.465425 22.767675 21.31109  21.873402 24.447159 22.875668 22.883732
 23.256102 22.321653 22.672398 21.633026 20.644176 23.265667 22.66707
 21.726377 24.586588 23.375206 23.452528 22.029839 23.133669 22.7542
 22.342123 23.423061 21.521685 23.21802  21.777098 24.749493 21.734793
 23.944794 22.244255 24.39961  23.554207 22.113003 23.756962 20.985195
 22.717316 23.393028 23.087238 22.595572 22.727722 21.354559 21.575428
 22.984032 22.571228 21.567768 21.619278 22.369814 22.166035 21.801172
 21.443169 22.850922 24.928234 22.670513 22.833815 22.157703 20.976486
 21.624157

## Backtest - Cruzamento de médias

### Gerar datas para os dados futuros

Neste trecho de código, a variável `ultima_data` é definida como a última data presente no índice dos dados.

Em seguida, a variável `datas_futuras` é criada utilizando a função `pd.date_range()`. Essa função gera um intervalo de datas começando no dia seguinte à `ultima_data` adicionando um dia (`ultima_data + pd.DateOffset(days=1)`) e com um número de períodos igual ao tamanho do vetor `previsoes_futuras`. O uso do método `normalize()` garante que todas as datas no intervalo sejam definidas como meia-noite, descartando informações de tempo.

Dessa forma, o código está criando um conjunto de datas futuras que correspondem às previsões geradas pelo modelo. Essas datas podem ser utilizadas para mapear as previsões futuras aos seus respectivos períodos de tempo, auxiliando na análise e visualização dos resultados.

In [58]:
ultima_data = pd.to_datetime(dados.index[-1])
datas_futuras = pd.date_range(start=ultima_data + pd.DateOffset(days=1), periods=len(previsoes_futuras)).normalize()

/tmp/ipykernel_7424/758396369.py:2: UserWarning: Discarding nonzero nanoseconds in conversion.
  datas_futuras = pd.date_range(start=ultima_data + pd.DateOffset(days=1), periods=len(previsoes_futuras)).normalize()


### Criar DataFrame para backtesting

Neste trecho de código, é criado um DataFrame chamado `dados_backtesting` com duas colunas: 'Date', que contém as datas futuras, e 'Close', que contém as previsões futuras.

Em seguida, o índice do DataFrame é definido como a coluna 'Date' usando o método `set_index('Date', inplace=True)`, que modifica o DataFrame atual, substituindo seu índice.

Por fim, são adicionadas as colunas 'Open', 'High' e 'Low' ao DataFrame `dados_backtesting`, e todos os valores dessas colunas são preenchidos com os valores da coluna 'Close'.

Essas etapas têm como objetivo criar um DataFrame que pode ser utilizado para realizar o backtesting de estratégias de negociação ou análises futuras com base nas previsões geradas.

In [59]:
dados_backtesting = pd.DataFrame({'Date': datas_futuras, 'Close': previsoes_futuras})
dados_backtesting.set_index('Date', inplace=True)
dados_backtesting['Open'] = dados_backtesting['High'] = dados_backtesting['Low'] = dados_backtesting['Close']

### Definir a estratégia de backtesting (cruzamento de médias)

Neste trecho de código, é definida a classe `MovingAverageCrossStrategy`, que herda da classe `Strategy`. Essa classe implementa uma estratégia de negociação baseada em cruzamento de médias móveis.

No método `__init__`, são definidos os parâmetros da estratégia, como o tamanho da janela da média móvel rápida (`fast_ma_window`) e da média móvel lenta (`slow_ma_window`). Além disso, a variável `buy_signal_triggered` é inicializada como `False`, indicando que não há sinal de compra acionado.

No método `next`, são realizadas as seguintes etapas:
- O histórico de preços de fechamento é obtido a partir dos dados.
- A média móvel rápida (`fast_ma`) é calculada utilizando os últimos `fast_ma_window` períodos.
- A média móvel lenta (`slow_ma`) é calculada utilizando os últimos `slow_ma_window` períodos.

Em seguida, são verificadas as condições de cruzamento das médias móveis. Se a média móvel rápida for maior que a média móvel lenta e o sinal de compra ainda não tiver sido acionado (`buy_signal_triggered == False`), é emitido um sinal de compra (`buy()`) e a variável `buy_signal_triggered` é atualizada para `True`.

Por outro lado, se a média móvel rápida for menor que a média móvel lenta e o sinal de compra tiver sido acionado anteriormente (`buy_signal_triggered == True`), é emitido um sinal de venda (`sell()`) e a variável `buy_signal_triggered` é atualizada para `False`.

Em resumo, essa classe implementa uma estratégia de cruzamento de médias móveis, onde um sinal de compra é acionado quando a média móvel rápida cruza acima da média móvel lenta, e um sinal de venda é acionado quando a média móvel rápida cruza abaixo da média móvel lenta.

In [60]:
class MovingAverageCrossStrategy(Strategy):
    def init(self):
        self.fast_ma_window = 50  # Janela da média móvel rápida
        self.slow_ma_window = 200  # Janela da média móvel lenta
        self.buy_signal_triggered = False

    def next(self):
        close_prices = self.data.Close
        # Cálculo da média móvel rápida
        fast_ma = close_prices[-self.fast_ma_window:].mean()
        # Cálculo da média móvel lenta
        slow_ma = close_prices[-self.slow_ma_window:].mean()

        if fast_ma > slow_ma and not self.buy_signal_triggered:
            self.buy()
            self.buy_signal_triggered = True
        elif fast_ma < slow_ma and self.buy_signal_triggered:
            self.sell()
            self.buy_signal_triggered = False

### Executar o backtesting

Neste código, é criada uma instância da classe `Backtest` chamada `bt`, que recebe os dados de backtesting `dados_backtesting` e a estratégia `MovingAverageCrossStrategy`.

Em seguida, o método `run()` é chamado para executar o backtest com a estratégia definida.

Por fim, o resultado do backtest é armazenado na variável `resultado` e é exibido usando `print(resultado)`.

Em resumo, esse código realiza um backtest usando a estratégia de cruzamento de médias móveis (`MovingAverageCrossStrategy`) nos dados de backtesting (`dados_backtesting`), e o resultado do backtest é exibido. O resultado pode conter informações como o saldo inicial, saldo final, número de negociações realizadas, entre outros.

In [61]:
bt = Backtest(dados_backtesting, MovingAverageCrossStrategy)
resultado = bt.run()

print(resultado)

Start                     1970-01-02 00:00:00
End                       1970-05-09 00:00:00
Duration                    127 days 00:00:00
Exposure Time [%]                    58.59375
Equity Final [$]                 10691.787811
Equity Peak [$]                  12073.483887
Return [%]                           6.917878
Buy & Hold Return [%]               -2.271362
Return (Ann.) [%]                   21.014897
Volatility (Ann.) [%]              160.612018
Sharpe Ratio                         0.130843
Sortino Ratio                        0.317348
Calmar Ratio                         1.266093
Max. Drawdown [%]                  -16.598225
Avg. Drawdown [%]                  -12.716464
Max. Drawdown Duration       34 days 00:00:00
Avg. Drawdown Duration       19 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       6.923568
Worst Trade [%]                      6.923568
Avg. Trade [%]                    